# Библиотеки

In [11]:
import nltk
import re
import numpy as np
nltk.download('punkt')
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Загрузка выборки

## Откроем и просмотрим файл

In [2]:
file = 'wine.data'

In [3]:
data = open(file).read().split('\n')
data.remove(data[-1]) # В data[-1] было ненужное нам значение ''
data

['1,14.23,1.71,2.43,15.6,127,2.8,3.06,.28,2.29,5.64,1.04,3.92,1065',
 '1,13.2,1.78,2.14,11.2,100,2.65,2.76,.26,1.28,4.38,1.05,3.4,1050',
 '1,13.16,2.36,2.67,18.6,101,2.8,3.24,.3,2.81,5.68,1.03,3.17,1185',
 '1,14.37,1.95,2.5,16.8,113,3.85,3.49,.24,2.18,7.8,.86,3.45,1480',
 '1,13.24,2.59,2.87,21,118,2.8,2.69,.39,1.82,4.32,1.04,2.93,735',
 '1,14.2,1.76,2.45,15.2,112,3.27,3.39,.34,1.97,6.75,1.05,2.85,1450',
 '1,14.39,1.87,2.45,14.6,96,2.5,2.52,.3,1.98,5.25,1.02,3.58,1290',
 '1,14.06,2.15,2.61,17.6,121,2.6,2.51,.31,1.25,5.05,1.06,3.58,1295',
 '1,14.83,1.64,2.17,14,97,2.8,2.98,.29,1.98,5.2,1.08,2.85,1045',
 '1,13.86,1.35,2.27,16,98,2.98,3.15,.22,1.85,7.22,1.01,3.55,1045',
 '1,14.1,2.16,2.3,18,105,2.95,3.32,.22,2.38,5.75,1.25,3.17,1510',
 '1,14.12,1.48,2.32,16.8,95,2.2,2.43,.26,1.57,5,1.17,2.82,1280',
 '1,13.75,1.73,2.41,16,89,2.6,2.76,.29,1.81,5.6,1.15,2.9,1320',
 '1,14.75,1.73,2.39,11.4,91,3.1,3.69,.43,2.81,5.4,1.25,2.73,1150',
 '1,14.38,1.87,2.38,12,102,3.3,3.64,.29,2.96,7.5,1.2,3,1547',
 

## Переведем данные в нужный нам формат(float)

In [4]:
data_list = []
for line in data:
    sent = []
    for word in line.split(','):
        floatword = float(word)
        sent.append(floatword) 
    data_list.append(sent)
data_list

[[1.0,
  14.23,
  1.71,
  2.43,
  15.6,
  127.0,
  2.8,
  3.06,
  0.28,
  2.29,
  5.64,
  1.04,
  3.92,
  1065.0],
 [1.0,
  13.2,
  1.78,
  2.14,
  11.2,
  100.0,
  2.65,
  2.76,
  0.26,
  1.28,
  4.38,
  1.05,
  3.4,
  1050.0],
 [1.0,
  13.16,
  2.36,
  2.67,
  18.6,
  101.0,
  2.8,
  3.24,
  0.3,
  2.81,
  5.68,
  1.03,
  3.17,
  1185.0],
 [1.0,
  14.37,
  1.95,
  2.5,
  16.8,
  113.0,
  3.85,
  3.49,
  0.24,
  2.18,
  7.8,
  0.86,
  3.45,
  1480.0],
 [1.0,
  13.24,
  2.59,
  2.87,
  21.0,
  118.0,
  2.8,
  2.69,
  0.39,
  1.82,
  4.32,
  1.04,
  2.93,
  735.0],
 [1.0,
  14.2,
  1.76,
  2.45,
  15.2,
  112.0,
  3.27,
  3.39,
  0.34,
  1.97,
  6.75,
  1.05,
  2.85,
  1450.0],
 [1.0,
  14.39,
  1.87,
  2.45,
  14.6,
  96.0,
  2.5,
  2.52,
  0.3,
  1.98,
  5.25,
  1.02,
  3.58,
  1290.0],
 [1.0,
  14.06,
  2.15,
  2.61,
  17.6,
  121.0,
  2.6,
  2.51,
  0.31,
  1.25,
  5.05,
  1.06,
  3.58,
  1295.0],
 [1.0,
  14.83,
  1.64,
  2.17,
  14.0,
  97.0,
  2.8,
  2.98,
  0.29,
  1.98,
  5.2,


# Извлекаем из данных признаки и классы

In [5]:
data_list = np.array(data_list)
# Class = data_list.transpose()[0]
# prizn = data_list.transpose()[1:]

# Генератор разбиений

In [6]:
kf = KFold(shuffle=True,random_state=42)

# Находим точность классификации

In [13]:
#scaler = MinMaxScaler(feature_range=(0, 1))
for k in range(1,51):
    accuracy = []
    neigh = KNeighborsClassifier(n_neighbors=k)
    for train, test in kf.split(data_list): #kf.split делит нашу выборку на обучающую и тестовую
        # Тренировка модели используется train
        X_train = data_list[train].transpose()[1:].transpose()
        Y_train = data_list[train].transpose()[0].transpose()
        # Масштабирование признаков
        X_train = preprocessing.scale(X_train)
        
        neigh.fit(X_train, Y_train)
        
        # Проверка точности модели используется test
        
        X_test = data_list[test].transpose()[1:].transpose()
        Y_test = data_list[test].transpose()[0].transpose()
        
        # Масштабирование признаков
        X_test = preprocessing.scale(X_test)
        
        y_pred = neigh.predict(X_test)
        
        result = accuracy_score(Y_test, y_pred)
        
        # Создаем массив результатов для каждого из сгенерированных разбиений(У нас их 5)
        accuracy.append(result)
    # Выводим среднее массива результатов для каждого k
    print('k =',k , 'accuracy =', np.mean(np.array(accuracy)))

k = 1 accuracy = 0.9496825396825397
k = 2 accuracy = 0.9326984126984128
k = 3 accuracy = 0.9496825396825397
k = 4 accuracy = 0.9384126984126985
k = 5 accuracy = 0.9496825396825397
k = 6 accuracy = 0.9495238095238095
k = 7 accuracy = 0.9607936507936508
k = 8 accuracy = 0.9495238095238095
k = 9 accuracy = 0.9607936507936508
k = 10 accuracy = 0.9663492063492063
k = 11 accuracy = 0.9720634920634922
k = 12 accuracy = 0.9553968253968254
k = 13 accuracy = 0.9609523809523809
k = 14 accuracy = 0.9609523809523809
k = 15 accuracy = 0.9609523809523809
k = 16 accuracy = 0.9552380952380952
k = 17 accuracy = 0.9553968253968254
k = 18 accuracy = 0.9498412698412698
k = 19 accuracy = 0.9553968253968254
k = 20 accuracy = 0.9607936507936508
k = 21 accuracy = 0.9609523809523809
k = 22 accuracy = 0.9550793650793651
k = 23 accuracy = 0.9720634920634922
k = 24 accuracy = 0.9663492063492063
k = 25 accuracy = 0.9720634920634922
k = 26 accuracy = 0.9663492063492063
k = 27 accuracy = 0.9665079365079364
k = 28 acc